In [ ]:
# =====================================================
# FAKE NEWS DETECTION – ALBERT-base + LoRA x FAKENEWSNET
# =====================================================

# 1. CÀI ĐẶT
!pip install -q transformers datasets peft accelerate bitsandbytes scikit-learn pandas numpy psutil sentencepiece

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType
from google.colab import drive

warnings.filterwarnings("ignore")

# 2. KIỂM TRA GPU
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"✅ Device: {device_name} | VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ CẢNH BÁO: Đang chạy trên CPU! Hãy chuyển sang T4 GPU.")

# 3. MOUNT DRIVE
drive.mount('/content/drive', force_remount=True)
OUTPUT_DIR = "/content/drive/MyDrive/FakeNewsNet_ALBERT_LoRA"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 4. TẢI DATASET FAKENEWSNET
print("\n⏳ Đang tải dataset FakeNewsNet...")
try:
    ds_gossip = load_dataset("rickstello/FakeNewsNet", "gossipcop", split="train")
    ds_politi = load_dataset("rickstello/FakeNewsNet", "politifact", split="train")
    dataset_full = concatenate_datasets([ds_gossip, ds_politi])
    df = pd.DataFrame(dataset_full)
except Exception as e:
    print(f"⚠️ Tải config thất bại ({e}), tải bản default...")
    dataset = load_dataset("rickstello/FakeNewsNet", split="train")
    df = pd.DataFrame(dataset)

print(f"Tổng số mẫu: {len(df)}")

# 5. XỬ LÝ CỘT (Auto-detect)
text_col = next((c for c in ['news_content', 'text', 'content', 'body'] if c in df.columns), None)
title_col = next((c for c in ['title', 'news_title', 'headline'] if c in df.columns), None)
label_col = next((c for c in ['real', 'label', 'class', 'fake'] if c in df.columns), None)

print(f"Mapping: Text='{text_col}' | Title='{title_col}' | Label='{label_col}'")
if not label_col: raise ValueError("❌ Không tìm thấy cột nhãn!")

df['label'] = df[label_col]

# Ghép Title + Text
title_data = df[title_col].fillna('') if title_col else ""
text_data = df[text_col].fillna('') if text_col else ""
df['content'] = title_data + " [SEP] " + text_data

# Làm sạch
def clean_text(s):
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r'https?://\S+', ' ', s)
    s = re.sub(r'<.*?>', ' ', s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

print("🧹 Đang làm sạch...", end="")
df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.len() > 20].drop_duplicates(subset=['content'])
print(f" → Sau xử lý: {len(df):,}")

# Class Weights
classes = np.unique(df['label'])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
class_weight_dict = {k: float(v) for k, v in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# 6. SPLIT DATA
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
})

# 7. TOKENIZER (ALBERT)
MODEL_NAME = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

print("⚙️ Tokenizing...")
tokenized = dataset_dict.map(tokenize_fn, batched=True, remove_columns=['content'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. MODEL & LoRA SETUP
print(f"\n⚙️ Tải model: {MODEL_NAME} và áp dụng LoRA...")
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Config nhãn
base_model.config.id2label = {0: "Fake", 1: "Real"}
base_model.config.label2id = {"Fake": 0, "Real": 1}

# Cấu hình LoRA cho ALBERT
# ALBERT dùng 'query', 'key', 'value' cho attention layers
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "key", "value"]
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# 9. CUSTOM TRAINER
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        w = torch.tensor(list(class_weight_dict.values()), dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=w)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 10. TRAINING ARGS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,                 
    per_device_train_batch_size=16,   
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-3,                 
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 11. TRAIN
print("\n🚀 BẮT ĐẦU HUẤN LUYỆN (ALBERT + LoRA)...")
trainer.train()

print("\n🎯 KẾT QUẢ TRÊN TEST SET:")
print(trainer.evaluate(tokenized["test"]))

# Save
final_path = os.path.join(OUTPUT_DIR, "final_albert_lora_fnn")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"\n✅ Đã lưu Adapter tại: {final_path}")

✅ Device: Tesla T4 | VRAM: 15.8 GB
Mounted at /content/drive

⏳ Đang tải dataset FakeNewsNet...


README.md:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

⚠️ Tải config thất bại (BuilderConfig 'gossipcop' not found. Available: ['default']), tải bản default...


FakeNewsNet.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/23196 [00:00<?, ? examples/s]

...

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 75,266 || all params: 11,760,388 || trainable%: 0.6400

🚀 BẮT ĐẦU HUẤN LUYỆN (ALBERT + LoRA)...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.710600,0.695486,0.242367,0.094564,0.058742,0.242367
2,0.701100,0.692970,0.757633,0.653160,0.574007,0.757633
3,0.700100,0.698380,0.757633,0.653160,0.574007,0.757633
4,0.697600,0.693883,0.757633,0.653160,0.574007,0.757633



🎯 KẾT QUẢ TRÊN TEST SET:


{'eval_loss': 0.692882239818573, 'eval_accuracy': 0.7576326914044152, 'eval_f1': 0.6531595570472056, 'eval_precision': 0.5740072950846979, 'eval_recall': 0.7576326914044152, 'eval_runtime': 2.019, 'eval_samples_per_second': 1054.477, 'eval_steps_per_second': 33.185, 'epoch': 4.0}

✅ Đã lưu Adapter tại: /content/drive/MyDrive/FakeNewsNet_ALBERT_LoRA/final_albert_lora_fnn
